In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(0)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/WganUnet_RC3')


dpath = DATA_PATH/(proj_id + '_1113_D.h5')
gpath = DATA_PATH/(proj_id + '_1113_G.h5')
keep_pct=1.0
bs=12
sz=256

c_lr=6e-4
c_lrs = np.array([c_lr,c_lr,c_lr,c_lr])

g_lr=1e-4
g_lrs = np.array([g_lr/10000,g_lr/100,g_lr])

torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=2).cuda()
netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=128, scale=16).cuda()
netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [6]:
trainer = WGANTrainer(netD=netD, netG=netG, md=md, bs=bs, sz=sz, dpath=dpath, gpath=gpath, genloss_fns=[FeatureLoss(multiplier=1e5)])
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer')

In [ ]:
#netG.freeze_to(1)
trainer.train(lrs_critic=c_lrs, lrs_gen=g_lrs, clr_critic=(1,8, 0.9, 0.9), clr_gen=(1,8, 0.9, 0.9), cycle_len=1, epochs=1)

 29%|██▊       | 30260/105697 [6:20:48<13:18:08,  1.58it/s]
WDist -306.2317810058594; RScore -212.06729125976562; FScore -518.299072265625; GAddlLoss 5267.126953125; GCount: 15130; GPenalty: 6.326979637145996; GCost: 499.3770446777344; ConPenalty: 96.62876892089844
 29%|██▊       | 30280/105697 [6:21:02<13:17:58,  1.58it/s]
WDist -274.03924560546875; RScore -204.10916137695312; FScore -478.1484069824219; GAddlLoss 5592.9921875; GCount: 15140; GPenalty: 10.330878257751465; GCost: 388.73394775390625; ConPenalty: 129.02268981933594
 29%|██▊       | 30300/105697 [6:21:16<13:20:54,  1.57it/s]
WDist -140.85427856445312; RScore -287.14910888671875; FScore -428.0033874511719; GAddlLoss 4472.87939453125; GCount: 15150; GPenalty: 11.350430488586426; GCost: 470.8115234375; ConPenalty: 158.89366149902344
 29%|██▊       | 30320/105697 [6:21:29<13:19:32,  1.57it/s]
WDist -116.58111572265625; RScore -373.71917724609375; FScore -490.30029296875; GAddlLoss 4567.39501953125; GCount: 15160; GPenalty: 12.